In [1]:
%load_ext dotenv
%dotenv
import os
from neo4j import GraphDatabase


AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(os.getenv("NEO4J_URL"), auth=(AUTH)) as driver:
    driver.verify_connectivity()

In [2]:
from importlib import reload
import openai_assistant

reload(openai_assistant)

<module 'openai_assistant' from '/Users/oskarhane/Development/tmp/neo4j-genai-design/openai_assistant.py'>

In [25]:
from custom_retrievers import movie
import retriever_router
import neo4j_retrievers
import neo4j_rag
import builtin_retrievers
reload(neo4j_retrievers)
reload(builtin_retrievers)
reload(movie)
reload(retriever_router)
reload(neo4j_rag)

llm = openai_assistant.OpenAIAssitant()

movie_plot_similarity = neo4j_retrievers.VectorRetriever(
    driver=driver,
    llm=llm, 
    name="movie_plot_similarity",
    index_name="moviePlotsEmbedding"
)
text2cypher_retriever = neo4j_retrievers.Text2Cypher(driver=driver, llm=llm)
movie_retriever = movie.MovieRetriever(driver=driver)

agentic_retriever = retriever_router.RetrieverRouter(
    llm=llm, 
    retrievers=[movie_plot_similarity, movie_retriever, text2cypher_retriever]
)

rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=agentic_retriever)
# or
single_retriever_rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=text2cypher_retriever)


In [20]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ["What year was the movie 'Casino' released?"]
Updated question: In what year was the movie 'Casino' released?
MovieRetriever invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Sharon Stone", "role": "Ginger McKenna"}, {"name": "James Woods", "role": "Lester Diamond"}]}]']
Unanswered questions: []
--- output ---
The movie "Casino" was released in 1995.
------ Single retriever ------
Text2Cypher invoked: What year was the movie "Casino" released?
MATCH (m:Movie {title: "Casino"}) RETURN m.year
[<Record m.year=1995>]
--- output ---
The movie "Casino" was released in 1995.


In [26]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('What movie is about "two mobster best friends and a trophy wife", and what year was it released? And who were some actors playing roles in it?')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('What movie is about "two mobster best friends and a trophy wife", and what year was it released? And who were some actors playing roles in it?')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ["What movie is about 'two mobster best friends and a trophy wife'?", "What year was the movie about 'two mobster best friends and a trophy wife' released?", "Who were some actors playing roles in the movie about 'two mobster best friends and a trophy wife'?"]
Routing question: What movie is about 'two mobster best friends and a trophy wife'?
Output: []
Updated question: What movie is about two mobster best friends and a trophy wife?
VectorRetriever invoked: two mobster best friends and a trophy wife
[-0.013337071053683758, -0.012353296391665936, 0.016347816213965416, -0.02626478485763073, -0.01087433472275734, 0.022791864350438118, 0.007474041543900967, 0.0006779952673241496, -0.0055130962282419205, -0.056992869824171066, 0.011554392985999584, -0.004067146219313145, 0.014050141908228397, 0.009038836695253849, 0.010418761521577835, -0.007480644155293703, 0.026027094572782516, -0.010121648199856281, 0.011263882741332054, -0.03132230788

In [18]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('How many actors were in the movie "Casino"')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('How many actors were in the movie "Casino"')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ["How many actors were in the movie 'Casino'?"]
Updated question: How many actors were in the movie 'Casino'?
MovieRetriever invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Sharon Stone", "role": "Ginger McKenna"}, {"name": "James Woods", "role": "Lester Diamond"}]}]']
Unanswered questions: []
--- output ---
The movie "Casino" had 4 actors.
------ Single retriever ------
Text2Cypher invoked: How many actors were in the movie "Casino"
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(a:Actor)
RETURN COUNT(DISTINCT a)
[<Record COUNT(DISTINCT a)=4>]
--- output ---
Based on the provided information, there were 4 actors in the movie "C

In [23]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('What are two the highest rated movies Robert De Niro has acted in? What are the plots in those movies?')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('What are two the highest rated movies Robert De Niro has acted in? What are the plots in those movies?')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ['What are the two highest rated movies Robert De Niro has acted in?', 'What is the plot in the first highest rated movie Robert De Niro has acted in?', 'What is the plot in the second highest rated movie Robert De Niro has acted in?']
Updated question: What are the two highest rated movies in which Robert De Niro has acted?
Text2Cypher invoked: What are the two highest rated movies in which Robert De Niro has acted?
MATCH (a:Actor {name: 'Robert De Niro'})-[:ACTED_IN]->(m:Movie)<-[r:RATED]-(:User)
RETURN m.title, AVG(r.rating) as AvgRating 
ORDER BY AvgRating DESC
LIMIT 2
[<Record m.title='Hide and Seek' AvgRating=4.5>, <Record m.title='Godfather: Part II, The' AvgRating=4.385185185185186>]
Response from tools: ['{"query": "MATCH (a:Actor {name: \'Robert De Niro\'})-[:ACTED_IN]->(m:Movie)<-[r:RATED]-(:User)\\nRETURN m.title, AVG(r.rating) as AvgRating \\nORDER BY AvgRating DESC\\nLIMIT 2", "records": [{"m.title": "Hide and Seek", "Av